In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-pbczbumc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-pbczbumc
  Resolved https://github.com/huggingface/transformers.git to commit f67dac97bdc63874f2288546b3fa87e69d2ea1c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7102716 sha256=7b6f87fe74f54c3a808fb57f578a2f67970713600ca5d16b3cfaaac3c7ccb861
  Store

In [ ]:
from torch.utils.data import Dataset, DataLoader
import glob
import os
from PIL import Image
import pandas as pd
import torch
import time
import json
from bert_score import score
from transformers import BlipProcessor, BlipForQuestionAnswering

device = "cuda" if torch.cuda.is_available() else "cpu"

"""save execution time"""
start_time = time.time()

In [ ]:
path = '/content/drive/MyDrive/VQA_RAD'
data_path = path + '/VQA_RAD Dataset Public.xlsx'
image_path = path + '/images'

In [ ]:
epochs = 1
each_n_epoch = 5

In [ ]:
class VQADataset(Dataset):
    def __init__(self, df_path, image_path, processor):
        self.df_path = df_path
        self.image_path = image_path
        self.processor = processor
        self.vqa_list = self.make_dict(df_path)

    def make_dict(self, df_path):
        df = pd.read_excel(df_path)
        questions = list(df['QUESTION'])
        answers = list(df['ANSWER'])
        images = [link.split('/')[-1] for link in list(df['IMAGEID'])]
        vqa_list = [None] * len(questions)
        idx = 0
        for im, q, ans in zip(images, questions, answers):
            vqa_list[idx] = [im, q, ans]
            idx += 1
        return vqa_list

    def __len__(self):
        return len(self.vqa_list)

    def __getitem__(self, idx):
        #print(self.vqa_list[idx])
        im_link, q, ans = self.vqa_list[idx]
        img_name = im_link.split('/')[-1]
        img_path = self.image_path + '/' + img_name
        img = Image.open(img_path).convert("RGB")
        encoding = self.processor(images=img, text=[q, str(ans)], padding="max_length", return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding['labels'] = encoding['input_ids'][1]
        encoding['input_ids'] = encoding['input_ids'][0]
        return encoding

In [ ]:
def collate_fn(batch):
   batch = list(filter(lambda x: x is not None, batch))
   return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

In [ ]:
"""train dataset"""
train_dataset = VQADataset(data_path, image_path, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
"""Training"""
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()

for epoch in range(epochs):
  print("Epoch:", epoch+1)
  train_loss = 0.0
  valid_loss = 0.0
  batch_num = 1
  for idx, batch in enumerate(train_dataloader):
    print("Batch: ", str(batch_num))
    batch_num +=1
    torch.cuda.empty_cache()
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)
    labels = batch.pop("labels").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=labels)

    loss = outputs.loss
    train_loss += loss.item() * input_ids.size(0)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

  # Compute the average loss for the training dataset
  train_loss /= len(train_dataloader.dataset)

Epoch: 1
Batch:  1
Batch:  2
Batch:  3
Batch:  4
Batch:  5
Batch:  6
Batch:  7
Batch:  8
Batch:  9
Batch:  10
Batch:  11
Batch:  12
Batch:  13
Batch:  14
Batch:  15
Batch:  16
Batch:  17
Batch:  18
Batch:  19
Batch:  20
Batch:  21
Batch:  22
Batch:  23
Batch:  24
Batch:  25
Batch:  26
Batch:  27
Batch:  28
Batch:  29
Batch:  30
Batch:  31
Batch:  32
Batch:  33
Batch:  34
Batch:  35
Batch:  36
Batch:  37
Batch:  38
Batch:  39
Batch:  40
Batch:  41
Batch:  42
Batch:  43
Batch:  44
Batch:  45
Batch:  46
Batch:  47
Batch:  48
Batch:  49
Batch:  50
Batch:  51
Batch:  52
Batch:  53
Batch:  54
Batch:  55
Batch:  56
Batch:  57
Batch:  58
Batch:  59
Batch:  60
Batch:  61
Batch:  62
Batch:  63
Batch:  64
Batch:  65
Batch:  66
Batch:  67
Batch:  68
Batch:  69
Batch:  70
Batch:  71
Batch:  72
Batch:  73
Batch:  74
Batch:  75
Batch:  76
Batch:  77
Batch:  78
Batch:  79
Batch:  80
Batch:  81
Batch:  82
Batch:  83
Batch:  84
Batch:  85
Batch:  86
Batch:  87
Batch:  88
Batch:  89
Batch:  90
Batch:  91

In [ ]:
image = Image.open("/content/drive/MyDrive/VQA_RAD/images/synpic42202.jpg.jpg").convert("RGB")

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/VQA_RAD/VQA_RAD Dataset Public.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
q = df[df['IMAGEID'] == 'https://medpix.nlm.nih.gov/images/full/synpic42202.jpg'].iloc[1]['QUESTION']
ans = df[df['IMAGEID'] == 'https://medpix.nlm.nih.gov/images/full/synpic42202.jpg'].iloc[1]['ANSWER']

In [ ]:
df[df['IMAGEID'] == 'https://medpix.nlm.nih.gov/images/full/synpic42202.jpg']

In [ ]:
ans

'Cardiovascular'

In [ ]:
q

'Which organ system is abnormal in this image?'

In [ ]:
inputs = processor(images=image, text=q, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values
input_ids = inputs.input_ids
generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
generated_caption

'yes'